In [1]:
#Load necessary libraries
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
sys.path.append('/Users/kristine/Library/Python/3.8/lib/python/site-packages')
import os
import ee
import geemap
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from ipyleaflet import Map, basemaps
import random
import json
import time
import ast

# # relative import for this folder hierarchy, credit: https://stackoverflow.com/a/35273613
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)

from wri_change_detection import preprocessing as npv
from wri_change_detection import gee_classifier as gclass
from wri_change_detection import post_classification_filters as pcf


In [2]:

#Initialize earth engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

Define seed number to reproduce results

In [3]:
num_seed = 30

Load Hansen forest change data and the DRC country boundary

In [4]:
# load Hansen forest change data
forestChange = ee.Image("UMD/hansen/global_forest_change_2019_v1_7")

# save projection information
projection_ee = forestChange.projection()
projection = projection_ee.getInfo()
crs = projection.get('crs')
crsTransform = projection.get('transform')
scale = projection_ee.nominalScale().getInfo()
print('CRS and Transform: ',crs, crsTransform)

# Load DRC feature
countryBoundaries = ee.FeatureCollection('projects/resource-watch-gee/gadm36_0')
DRCBoundary = countryBoundaries.filterMetadata('GID_0','equals','COD').first().geometry()


CRS and Transform:  EPSG:4326 [0.00025, 0, -180, 0, -0.00025, 80]


Mask tree cover loss to areas with greater than 30% tree cover in 2000 and in primary forest areas.

In [5]:
# load primary forest data
primaryForest = ee.Image('UMD/GLAD/PRIMARY_HUMID_TROPICAL_FORESTS/v1/2001')

# select tree cover loss band and unmask with value 0 to represent no loss
tclYear = forestChange.select(['lossyear']).unmask(0)

# define mask of primary forest areas
primaryForestValid = primaryForest.eq(1);
# define mask for where Hansen data is valid
forestChangeValid = forestChange.select('datamask').eq(1)
# define mask of tree cover loss > 30%
forestCoverValid = forestChange.select('treecover2000').gte(30)

# create final mask where (primaryForestValid = 1) AND (forestChangeValid = 1) AND (forestCoverValid = 1)
treeCoverLossValid = primaryForestValid.bitwiseAnd(forestCoverValid).bitwiseAnd(forestChangeValid)

# update tree cover loss year to mask invalid areas
tclYearMasked = tclYear.updateMask(treeCoverLossValid)
# now tclYearMasked = {0 when tree cover loss did not occur, 1-19 for year when tree cover loss occured} only in valid areas


Plot masked tree cover loss to validate.

In [6]:
center = [-1.776125,23.710125]
zoom = 4
tclPalette = ['#0051ff',
            '#f7f4f9','#f7f4f9',
            '#e7e1ef','#e7e1ef',
            '#d4b9da','#d4b9da',
            '#c994c7','#c994c7',
            '#df65b0','#df65b0',
            '#e7298a','#e7298a',
            '#ce1256','#ce1256',
            '#980043','#980043',
            '#67001f','#67001f']
tclViz = {'min': 0, 'max': 19, 'palette': tclPalette}
Map1 = geemap.Map(center=center, zoom=zoom,basemap=basemaps.Esri.WorldImagery,add_google_map = False)
Map1.addLayer(forestChangeValid.updateMask(forestChangeValid),
              {'min': 0, 'max': 1, 'palette': ['#f9261b','#5a1bf9']},name='Hansen Valid')
Map1.addLayer(forestCoverValid.updateMask(forestCoverValid),
              {'min': 0, 'max': 1, 'palette': ['#f7f91b','#28ce4c']},name='Tree Cover > 30')
Map1.addLayer(primaryForestValid.updateMask(primaryForestValid),
              {'min': 0, 'max': 1, 'palette': ['#f9801b','#9d1bf9']},name='Primary Forest')
Map1.addLayer(forestChange.select(['lossyear']),tclViz,name='Tree Cover Loss UnMasked')
Map1.addLayer(tclYearMasked,tclViz,name='Tree Cover Loss Masked')

display(Map1)

Map(center=[-1.776125, 23.710125], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleB…

Create three binary layers:
1. Tree cover loss that occurred from 2001 through 2011, this is referred to as early loss, which we'll use to calculate the distance to previous loss
2. Tree cover loss that occurred from 2018 through 2019, this is loss we wil use in evaluating our model
3. Tree cover loss that occurred from 2012 through 2017, this is our reference loss which we will be modelling

In [7]:
# Define mask when tree cover occurred from 2001 through 2011
tclEarlyLoss = tclYearMasked.expression(
    '(year>0 && year<12)', {
      'year': tclYearMasked.select('lossyear')
})
# Define mask when tree cover occurred after 2017
tclLaterLoss = tclYearMasked.expression(
    '(year>17)', {
      'year': tclYearMasked.select('lossyear')
})

# Define binary variable for tree cover loss that occurred from 2012 through 2017,
# loss that occurred after 2017 is marked as 0
tclReferenceLoss = tclYearMasked.expression(
    '(year>11 && year<18) ? 1 : 0', {
      'year': tclYearMasked.select('lossyear')
})

# Define mask for tree cover loss that occurred from 2012 through 2017
referenceTreeCoverLossValid = treeCoverLossValid.bitwiseAnd(tclEarlyLoss.eq(0))

# Mask tree cover loss years to get binary 0 for no tree cover loss and 1 for 
tclReferenceLoss = tclReferenceLoss.updateMask(referenceTreeCoverLossValid).gt(0)
tclReferenceLoss = tclReferenceLoss.rename('loss')

# Map layers to double check!
Map2 = geemap.Map(center=center, zoom=zoom,basemap=basemaps.Esri.WorldImagery,add_google_map = False)
Map2.addLayer(tclYearMasked,tclViz,name='Tree Cover Loss Masked')
Map2.addLayer(tclEarlyLoss,{'min': 0, 'max': 1, 'palette': ['#f9261b','#5a1bf9']},name='Early Loss')
Map2.addLayer(tclLaterLoss,{'min': 0, 'max': 1, 'palette': ['#f9261b','#5a1bf9']},name='Later Loss')
Map2.addLayer(tclReferenceLoss,{'min': 0, 'max': 1, 'palette': ['#f9261b','#5a1bf9']},name='Historical Loss')
display(Map2)


Map(center=[-1.776125, 23.710125], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleB…

Gather training, testing, and validation points

In [8]:
#We'll remove training points that are within a certain distance to the validation and test points
distanceToFilter = 10000
numPoints = 6000

#Sample point locations that we will split into training, validation, and test sets
point_locations = npv.getStratifiedSampleBandPoints(tclReferenceLoss, region=DRCBoundary, 
                                                       numPoints=numPoints, bandName='loss',seed=num_seed,
                                                       geometries=True, projection=projection_ee)

#Assign random value between 0 and 1 to split into training, validation, and test sets
point_locations = point_locations.randomColumn(columnName='TrainingSplit', seed=num_seed)
#First split training set from the rest, taking 70% of the points for training
#Roughly 70% training, 30% for validation + testing
training_split = 0.7
training_locations = point_locations.filter(ee.Filter.lt('TrainingSplit', training_split))
validation_and_test = point_locations.filter(ee.Filter.gte('TrainingSplit', training_split))

#Define distance filter to remove training points within a certain distance of test points and validation points
distFilter = ee.Filter.withinDistance(distance=distanceToFilter, leftField='.geo', rightField= '.geo', maxError= 1)
join = ee.Join.inverted()
training_locations = join.apply(training_locations, validation_and_test, distFilter);

#Assign another random value between 0 and 1 to validation_and_test to split to validation and test sets
validation_and_test = validation_and_test.randomColumn(columnName='ValidationSplit', seed=num_seed)
#Of the 30% saved for validation + testing, half goes to validation and half goes to test
#Meaning original sample will be 70% training, 15% validation, 15% testing
validation_split = 0.5 
validation_locations = validation_and_test.filter(ee.Filter.lt('ValidationSplit', validation_split))
test_locations = validation_and_test.filter(ee.Filter.gte('ValidationSplit', validation_split))

#Apply distance filter to remove validation points within a certain distance of test points
validation_locations = join.apply(validation_locations, test_locations, distFilter);

#Export these locations to an Earth Engine asset
location_description = '{}_locations'
location_assetID = 'users/listerkristineanne/CongoDeforestation/TrainingPoints_{}_locations'
#location_assetID = 'users/listerkristineanne/CongoDeforestation/TrainingPoints/TrainingPoints_{}_locations'

# export_results_task = ee.batch.Export.table.toAsset(
#     collection=training_locations, 
#     description = location_description.format('training'), 
#     assetId = location_assetID.format('training'))
# export_results_task.start()
    
# export_results_task = ee.batch.Export.table.toAsset(
#     collection=validation_locations, 
#     description = location_description.format('validation'), 
#     assetId = location_assetID.format('validation'))
# export_results_task.start()

# export_results_task = ee.batch.Export.table.toAsset(
#     collection=test_locations, 
#     description = location_description.format('test'), 
#     assetId = location_assetID.format('test'))
# export_results_task.start()
 
# #Wait for last export to finish
# while export_results_task.active():
#     print('Polling for task (id: {}).'.format(export_results_task.id))
#     time.sleep(30)
# print('Done with export.')

Define predictor variables

In [9]:
# Load elevation
elevation = ee.Image("CGIAR/SRTM90_V4")
# Calculate slope
slope = ee.Terrain.slope(elevation)

# Load roads data
roads = ee.FeatureCollection('users/listerkristineanne/CongoDeforestation/PredictorVariables/HistoricalFinal/RDC_voirie')

# Create cost map for measuring distance, for now we'll list crossing any pixel as the same weight
# Generate a constant image with value 1, clip it to the bounds of the DRC
cost = ee.Image.constant(1).clip(DRCBoundary.bounds())
maxDistance=50000

# Calculate distance to loss from 2001 to 2011
# First unmask earlyLoss to get target pixels
earlyLossUnmasked = tclEarlyLoss.unmask(0)
distanceToEarlyLoss = cost.cumulativeCost(source= earlyLossUnmasked, maxDistance=maxDistance).rename('earlyLossDistance')
distanceToEarlyLoss = distanceToEarlyLoss.unmask(maxDistance)

# Burn roads feature collection to image
roads = roads.map(lambda x: x.set({'constant':1}))
roadsImage = roads.reduceToImage(['constant'], ee.Reducer.first())
# Convert image to binary 0:1 if there is a road
roadsImage = roadsImage.unmask(0)
# Calculate distance to roads
distanceToRoads = cost.cumulativeCost(source= roadsImage, maxDistance=maxDistance).rename('roadsDistance')
distanceToRoads = distanceToRoads.unmask(maxDistance)

# Load rural complex
ruralComplex = ee.Image('users/listerkristineanne/CongoDeforestation/PredictorVariables/HistoricalFinal/DRC_2010_Rural_Complex')
# Calculate distance to rural complex
distanceToRuralComplex = cost.cumulativeCost(source= ruralComplex, maxDistance=maxDistance).rename('ruralComplexDistance')
distanceToRuralComplex = distanceToRuralComplex.unmask(maxDistance)

# Load protected areas
protectedAreas = ee.FeatureCollection('users/listerkristineanne/CongoDeforestation/PredictorVariables/HistoricalFinal/RDC_aire_protegee')
# Filter hunting areas to another feature collection
huntingAreas = protectedAreas.filterMetadata('REGLEMENT','equals','Domaine de chasse')
# Filter protected areas to remove hunting areas
protectedAreasFiltered = protectedAreas.filterMetadata('REGLEMENT','not_equals','Domaine de chasse')

# Burn protected areas feature collection to image
protectedAreasFiltered = protectedAreasFiltered.map(lambda x: x.set({'constant':1}))
protectedAreasImage = protectedAreasFiltered.reduceToImage(['constant'], ee.Reducer.first())
# Convert image to binary 0:1 if there is a road
protectedAreasImage = protectedAreasImage.unmask(0)
# Calculate distance to roads
distanceToProtectedAreas = cost.cumulativeCost(source= protectedAreasImage, maxDistance=maxDistance).rename('protectedAreasDistance')
distanceToProtectedAreas = distanceToProtectedAreas.unmask(maxDistance)


# Burn hunting areas feature collection to image
huntingAreas = huntingAreas.map(lambda x: x.set({'constant':1}))
huntingAreasImage = huntingAreas.reduceToImage(['constant'], ee.Reducer.first())
# Convert image to binary 0:1 if there is a road
huntingAreasImage = huntingAreasImage.unmask(0)
# Calculate distance to roads
distanceToHuntingAreas = cost.cumulativeCost(source= huntingAreasImage, maxDistance=maxDistance).rename('huntingAreasDistance')
distanceToHuntingAreas = distanceToHuntingAreas.unmask(maxDistance)

# Define list of predictor variable images and convert to a single image
predictor_variable_list = [distanceToEarlyLoss,distanceToRuralComplex,distanceToProtectedAreas,
                           distanceToHuntingAreas,
                           distanceToRoads,elevation,slope]
predictor_variable_image = ee.ImageCollection(predictor_variable_list).toBands()

# Rename bands
predictor_variable_names = ['earlyLossDistance','ruralComplexDistance','roadsDistance',
                            'protectedAreasDistance','huntingAreasDistance','elevation','slope']
predictor_variable_image = predictor_variable_image.rename(predictor_variable_names)



In [10]:
#Old distance
# roads = ee.FeatureCollection('users/listerkristineanne/CongoDeforestation/PredictorVariables/HistoricalFinal/RDC_voirie')

# # Define kernel to calculate distance, we'll only calculate distance for features withint 500km
# kernel = ee.Kernel.euclidean(500000,"meters")

# # Calculate distance to loss from 2001 to 2011
# distanceToEarlyLoss = tclEarlyLoss.unmask(0).distance(kernel,True).clip(DRCBoundary).rename('earlyLossDistance')

# # Burn roads feature collection to image
# roadsImage = roads.reduceToImage(['long_m'], ee.Reducer.first())
# # Convert image to binary 0:1 if there is a road
# roadsImage = roadsImage.gt(0)
# # Calculate distance to roads
# distanceToRoads = roadsImage.distance(kernel,True).clip(DRCBoundary).rename('roadsDistance')

# # Define list of predictor variable images and convert to a single image
# predictor_variable_list = [distanceToEarlyLoss,
#                            distanceToRoads]
# predictor_variable_image = ee.ImageCollection(predictor_variable_list).toBands()

# # Rename bands t
# predictor_variable_names = ['earlyLossDistance','roadsDistance']
# predictor_variable_image = predictor_variable_image.rename(predictor_variable_names)



Sample predictor variables at training, validation, and test points 

In [11]:
#Define properties to export to an Earth Engine asset
points_description = '{}_points'
points_assetID = 'users/listerkristineanne/CongoDeforestation/TrainingPoints_{}_points'

# Load the point locations assets
training_locations_asset = ee.FeatureCollection(location_assetID.format('training'))
validation_locations_asset = ee.FeatureCollection(location_assetID.format('validation'))
test_locations_asset = ee.FeatureCollection(location_assetID.format('test'))

# Sample predictor variable at location
training_points = predictor_variable_image.sampleRegions(training_locations_asset, 
                                                         projection=projection_ee, geometries=True,tileScale=16)
validation_points = predictor_variable_image.sampleRegions(validation_locations_asset, 
                                                           projection=projection_ee, geometries=True,tileScale=16)
test_points = predictor_variable_image.sampleRegions(test_locations_asset, 
                                                     projection=projection_ee, geometries=True,tileScale=16)

# # # Export results
# export_results_task = ee.batch.Export.table.toAsset(
#     collection=training_points, 
#     description = points_description.format('training'), 
#     assetId = points_assetID.format('training'))
# export_results_task.start()

# export_results_task = ee.batch.Export.table.toAsset(
#     collection=validation_points, 
#     description = points_description.format('validation'), 
#     assetId = points_assetID.format('validation'))
# export_results_task.start()

# export_results_task = ee.batch.Export.table.toAsset(
#     collection=test_points, 
#     description = points_description.format('test'), 
#     assetId = points_assetID.format('test'))
# export_results_task.start()

# #Wait for last export to finish
# while export_results_task.active():
#     print('Polling for task (id: {}).'.format(export_results_task.id))
#     time.sleep(30)
# print('Done with export.')

Define model parameters to test

In [12]:
#Define dictionaries of parameters and models to test
#You can find the inputs for the parameters under the ee.Classifiers section of GEE

rf_parameters = {'seed':[num_seed], 
          'numberOfTrees': [50,100], 
          'variablesPerSplit': [1,2,None], 
          'minLeafPopulation': [4,10,50], 
          'bagFraction': [None,0.5,.3], 
          'maxNodes': [None, 20, 50]
         }
#buildGridSearchList converts the parameter dictionary into a list of classifiers that can be used in cross-validation
rf_classifier_list = gclass.buildGridSearchList(rf_parameters,'smileRandomForest')

svm_parameters = {'decisionProcedure':[None]}
svm_classifier_list = gclass.buildGridSearchList(svm_parameters,'libsvm')

#maxent_parameters = {'minIterations':[10,100],'maxIterations':[50,200]}
#maxent_classifier_list = gclass.buildGridSearchList(maxent_parameters,'gmoMaxEnt')

classifier_list = rf_classifier_list+svm_classifier_list#+maxent_classifier_list

Use cross-validation to test models

In [13]:

#Name y_column
y_column = 'loss'

#Define assetId and description format to export to GEE
cv_results_assetId = 'users/listerkristineanne/CongoDeforestation/TrainingPoints/CV_Results_20210305'
cv_results_description = 'cv_export_20210305'

#Load training points
training_points = ee.FeatureCollection(points_assetID.format('training'))

#Perform cross validation, returns a feature collection
cv_results = gclass.kFoldCrossValidation(inputtedFeatureCollection = training_points, 
                                     propertyToPredictAsString = y_column, 
                                     predictors = predictor_variable_names, 
                                     listOfClassifiers = classifier_list,
                                     k=3,seed=num_seed)
#Export results to GEE
export_results_task = ee.batch.Export.table.toAsset(
        collection=cv_results, 
        description = cv_results_description, 
        assetId = cv_results_assetId)
export_results_task.start()

#Wait for export to finish
while export_results_task.active():
    print('Polling for task (id: {}).'.format(export_results_task.id))
    time.sleep(30)
print('Done with export.')

Number of Sample Points 3417
Average Fold Size 1139
Polling for task (id: R7WQUY3CN2NNPPTYKKXJUOJD).
Done with export.


Evaluate accuracy of models on validation set

In [14]:
#Create empty dataframe to save results of cross validation
accuracy_and_keys = pd.DataFrame()

#Load cross-validation results
results = ee.FeatureCollection(cv_results_assetId)
#Get the best result by the cross validation score
best_result = results.sort('Validation Score', False).first()

#Load params as a dictionary
params = best_result.get('Params').getInfo()
params = ast.literal_eval(params)

#Get the calssifier name
classifierName = best_result.get('Classifier Type').getInfo()

#Load classifier with best params
best_model = gclass.defineClassifier(params,classifierName)

#Load training and validation points
training_points = ee.FeatureCollection(points_assetID.format('training'))
validation_points = ee.FeatureCollection(points_assetID.format('validation'))

#Train a classifier with the best params on the training data
best_model = best_model.train(training_points, classProperty=y_column, 
                              inputProperties=predictor_variable_names, subsamplingSeed=num_seed)

#Predict over validation data
validation_points_predicted = validation_points.classify(best_model)

#Get confusion matrix and accuracy score
confusion_matrix = validation_points_predicted.errorMatrix(y_column, 'classification');
accuracy = confusion_matrix.accuracy().getInfo()


#Get confusion matrix and accuracy score
confusion_matrix = validation_points_predicted.errorMatrix(y_column, 'classification');
accuracy = confusion_matrix.accuracy().getInfo()
print('--------------------------------------------------------------')
print('Final Model Validation Set Confusion Matrix')
print(gclass.pretty_print_confusion_matrix_binary(confusion_matrix.getInfo()))
print('Final Model Validation Set Accuracy',accuracy)
print('\n')


# #Predict over test data
# test_points = ee.FeatureCollection(points_assetID.format('test'))

# test_points_predicted = test_points.classify(best_model)

# #Get confusion matrix and accuracy score
# confusion_matrix = test_points_predicted.errorMatrix(y_column, 'classification');
# accuracy = confusion_matrix.accuracy().getInfo()
# print('--------------------------------------------------------------')
# print('Final Model Test Set Confusion Matrix')
# print(gclass.pretty_print_confusion_matrix_binary(confusion_matrix.getInfo()))
# print('Final Model Test Set Accuracy',accuracy)
# print('\n')

--------------------------------------------------------------
Final Model Validation Set Confusion Matrix
                Predicted_False  Predicted_True
_                                              
Observed_False              486              81
Observed_True                67             377
Final Model Validation Set Accuracy 0.8536102868447082




In [21]:
print(best_model.schema().getInfo())

['earlyLossDistance', 'ruralComplexDistance', 'roadsDistance', 'protectedAreasDistance', 'huntingAreasDistance', 'elevation', 'slope']


In [19]:
#Use best model and predictor variable image to predict loss
predicted_loss = predictor_variable_image.updateMask(referenceTreeCoverLossValid).classify(best_model)

# Map layers to double check!
Map3 = geemap.Map(center=center, zoom=zoom,basemap=basemaps.Esri.WorldImagery,add_google_map = False)
#Map3.addLayer(tclYearMasked,tclViz,name='Tree Cover Loss Masked')
Map3.addLayer(tclReferenceLoss,{'min': 0, 'max': 1, 'palette': ['#98BC47','#DA6E9A']},name='Historical Loss')
Map3.addLayer(predicted_loss,{'min': 0, 'max': 1, 'palette': ['#98BC47','#DA6E9A']},name='Predicted Loss')
#Map3.addLayer(tclReferenceLoss.eq(predicted_loss),{'min': 0, 'max': 1, 'palette': ['#98BC47','#DA6E9A']},name='Accurate Prediction')

display(Map3)

Map(center=[-1.776125, 23.710125], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleB…

In [16]:
# bucket = 'wri-congo-deforestation'
# fileFormat = 'CSV'

# #Export.table.toCloudStorage(collection, description, bucket, fileNamePrefix, fileFormat, selectors)

# # # Export results
# export_results_task = ee.batch.Export.table.toCloudStorage(
#     collection=training_points, 
#     description = points_description.format('training'), 
#     bucket = bucket,
#     fileFormat = fileFormat,
#     fileNamePrefix = points_description.format('training'))
# export_results_task.start()

# export_results_task = ee.batch.Export.table.toCloudStorage(
#     collection=training_points, 
#     description = points_description.format('validation'), 
#     bucket = bucket,
#     fileFormat = fileFormat,
#     fileNamePrefix = points_description.format('validation'))
# export_results_task.start()

# export_results_task = ee.batch.Export.table.toCloudStorage(
#     collection=training_points, 
#     description = points_description.format('test'), 
#     bucket = bucket,
#     fileFormat = fileFormat,
#     fileNamePrefix = points_description.format('test'))
# export_results_task.start()

# #Wait for last export to finish
# while export_results_task.active():
#     print('Polling for task (id: {}).'.format(export_results_task.id))
#     time.sleep(30)
# print('Done with export.')